In [1]:
from jupyter_dash import JupyterDash

import dash
import dash_leaflet as dl
from dash import dcc, html
import plotly.express as px
from dash import dash_table as dt
from dash.dependencies import Input, Output, State
from datetime import datetime

import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps

# Module need it to decode images or text
import base64

#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from animalsCRUD import AnimalShelter





###########################
# Data Manipulation / Model
###########################
# FIX ME change for your username and password and CRUD Python module name
username = "aacuser"
password = "aacpass"
shelter = AnimalShelter(username, password)


# class read method must support return of cursor object 
df = pd.DataFrame.from_records(shelter.read({}))



#########################
# Dashboard Layout / View
#########################
app = JupyterDash("SimpleExample")

#FIX ME Add in Grazioso Salvare’s logo
image_filename = 'gs-logo.png' #image file name and type
encoded_image = base64.b64encode(open(image_filename, 'rb').read()).decode('utf-8')

# radio btn filtering option
filter_option = ['Water Rescue', 'Mountain or Wilderness Rescue', 'Disaster or Individual Tracking', 'Reset']

# app loyout elements configuration
app.layout = html.Div([
    # rendering img
    html.Center(html.Img(src=f'data:image/png;base64,{encoded_image}', style={'width': '200px', 'height':'200px'})),
    # dashboard title 
    html.Center(html.B(html.H1("Grazioso Salvare Dashboard"))),
    # display info update time
    html.Center(html.P(f"Update: {datetime.now().strftime('%Y-%m-%d')}", style={'font-size': '14px', 'color': 'gray'})),
    # my name tag
    html.Center(html.H4("Angel Baez")),
    html.Hr(),
    
#FIXME Add in code for the interactive filtering options. For example, Radio buttons, drop down, checkboxes, etc.
    # radio button for filter values 
    dcc.RadioItems(
        id='filter-type', # filter id name 
        options=[{'label': i, 'value': i} for i in filter_option], value=None,
        labelStyle={'display': 'inline-block'}
    ),

    html.Hr(),
    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
#FIXME: Set up the features for your interactive data table to make it user-friendly for your client
#If you completed the Module Six Assignment, you can copy in the code you created here 
        
        filter_action="native", # filtering option
        sort_action="native", # sort docs option
        page_action="native", # 
        page_current=0, # initial page
        page_size=10, # documents per page
        row_selectable="multi", # multiple selection
        selected_rows=[], # no row selected when table starts
        selected_columns=[], # no column selected  table starts
        style_table={'overflowX': 'auto'}, # if table does not fit in the screen will add a scroll bar
        style_cell={'textAlign': 'left'}, # alihn context to left
    ),
    
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',
        ),
        html.Div(
            html.Div(id='map-id', className='col s12 m6'),
        )       
    ]),
])

#############################################
# Interaction Between Components / Controller
#############################################




    
@app.callback([Output('datatable-id', 'data'), 
               Output('datatable-id', 'columns')],
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):
## FIX ME Add code to filter interactive data table with MongoDB queries
    # conditions for filtering options
    if filter_type == 'Reset':
        dff = df
    elif filter_type == 'Water Rescue':
        dff = df[df['breed'].isin(['Labrador Retriever Mix', 'Chesapeake Bay Retriever', 'Newfoundland'])]
    elif filter_type == 'Mountain or Wilderness Rescue':
        dff = df[df['breed'].isin(['German Shepherd', 'Alaskan Malamute', 'Old English Sheepdog', 'Siberian Husky', 'Rottweiler'])]
    elif filter_type == 'Disaster or Individual Tracking':
        dff = df[df['breed'].isin(['Doberman Pinscher', 'German Shepherd', 'Golden Retriever', 'Bloodhound', 'Rottweiler'])]
    else:
        dff = df
    
    columns = [{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
    data = dff.to_dict('records')

    return data, columns





@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")]
)
def update_graphs(viewData):
    dff = pd.DataFrame.from_dict(viewData)
    ###FIX ME ####
    # add code for chart of your choice (e.g. pie chart) #
    if dff.empty:
        return []
    df_grouped = dff.groupby(['breed']).size().reset_index(name='Count')
    fig = px.pie(df_grouped, values='Count', names='breed', title="Animal Types")
    return [dcc.Graph(figure=fig)]

@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")]
)
def update_map(viewData):
#FIXME: Add in the code for your geolocation chart
#If you completed the Module Six Assignment, you can copy in the code you created here.
    dff = pd.DataFrame.from_dict(viewData)
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id")
        ] + [
            dl.Marker(position=[row['location_lat'], row['location_long']], children=[
                dl.Tooltip(row['breed']),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(row['name'])
                ])
            ]) for index, row in dff.iterrows()
        ])
    ]

app


IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)

C:\Users\anedb\OneDrive\Desktop\mongoDB\ProjectTwo\lib\site-packages\dash\dash.py:579: UserWarning:

JupyterDash is deprecated, use Dash instead.
See https://dash.plotly.com/dash-in-jupyter for more details.



In [2]:
app.run_server(debug=True)

Dash app running on http://127.0.0.1:8050/
